# 🏦 NeoScore - Data Cleaning & Upload to BigQuery

**Autor**: Luca Camus  
**Fecha**: Enero 2026  
**Objetivo**: Limpiar datos transaccionales y cargarlos a BigQuery

## 1. Instalación y Configuración

In [ ]:
# Instalar dependencias
!pip install google-cloud-bigquery pyarrow pandas-gbq --quiet

In [ ]:
# Autenticación con Google Cloud
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import numpy as np
from google.cloud import bigquery
import re

# Configuración del proyecto
PROJECT_ID = "scoring-bancario"
DATASET_ID = "analisis_bancario"
TABLE_ID = "scoring_transacciones"

print("✅ Dependencias instaladas y autenticación completa")

## 2. Cargar Datos desde Google Drive

In [ ]:
# Montar Google Drive para acceder al CSV
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
# IMPORTANTE: Ajusta esta ruta según donde tengas tu archivo
# ============================================================
CSV_PATH = '/content/drive/MyDrive/Finanzas/Proyectos/Scoring bancario/archive (2)/bank_transactions.csv'

print('✅ Google Drive montado correctamente')

In [ ]:
# Cargar el CSV desde Google Drive
df = pd.read_csv(CSV_PATH)
print(f"📊 Dataset cargado: {df.shape[0]:,} filas x {df.shape[1]} columnas")
print(f"\n📋 Columnas originales:\n{df.columns.tolist()}")

In [ ]:
# Vista previa de los datos
df.head(10)

In [ ]:
# Información del dataset
df.info()

## 3. Función de Sanitización de Nombres de Columnas

In [ ]:
def sanitize_column_names(df):
    """
    Convierte nombres de columnas a snake_case compatible con BigQuery.
    - Elimina paréntesis y caracteres especiales
    - Convierte espacios a guiones bajos
    - Convierte a minúsculas
    """
    new_columns = {}
    for col in df.columns:
        # Remover paréntesis y su contenido opcional
        new_name = re.sub(r'\s*\([^)]*\)', '', col)
        # Reemplazar espacios por guiones bajos
        new_name = new_name.replace(' ', '_')
        # Convertir a minúsculas
        new_name = new_name.lower()
        # Remover caracteres especiales restantes
        new_name = re.sub(r'[^a-z0-9_]', '', new_name)
        # Remover guiones bajos dobles
        new_name = re.sub(r'_+', '_', new_name)
        # Remover guiones bajos al inicio/final
        new_name = new_name.strip('_')
        new_columns[col] = new_name
    
    df = df.rename(columns=new_columns)
    return df

# Aplicar sanitización
df = sanitize_column_names(df)
print(f"\n✅ Columnas sanitizadas:\n{df.columns.tolist()}")

## 4. Limpieza de Datos

### Problemas identificados en el CSV:
1. **Fechas zombi**: `1/1/1800` usado como placeholder para datos faltantes
2. **String 'nan'**: Algunos valores aparecen como texto 'nan' en vez de nulo
3. **Género nulo**: Campos vacíos que debemos mantener sin sesgar
4. **Balance vacío**: Algunos registros sin balance
5. **Años futuros**: Fechas como '94' se interpretan como 2094

In [ ]:
print("\n🔧 Iniciando limpieza de datos...")
total_rows = len(df)

# 4.1 Limpiar CustomerDOB - Reemplazar fechas zombi por None
# Detectado: líneas 18, 24, 30, 37, etc. tienen '1/1/1800'
zombie_dates = ['1/1/1800', '01/01/1800', 'nan', 'NaN', 'NaT', '']
df['customerdob'] = df['customerdob'].replace(zombie_dates, None)
df['customerdob'] = df['customerdob'].replace('nan', None)

# Convertir a datetime donde sea posible
df['customerdob'] = pd.to_datetime(df['customerdob'], format='%d/%m/%y', errors='coerce')

# Corregir años interpretados incorrectamente (ej: 94 -> 2094 debería ser 1994)
# Si la fecha es futura, restar 100 años
mask_future = df['customerdob'] > pd.Timestamp.now()
df.loc[mask_future, 'customerdob'] = df.loc[mask_future, 'customerdob'] - pd.DateOffset(years=100)

dob_nulls = df['customerdob'].isna().sum()
dob_pct = (dob_nulls / total_rows) * 100
print(f"   - Fechas zombi limpiadas. Nulos en DOB: {dob_nulls:,} ({dob_pct:.2f}% del total) ✅ Aceptable")

In [ ]:
# 4.2 Limpiar TransactionDate
df['transactiondate'] = pd.to_datetime(df['transactiondate'], format='%d/%m/%y', errors='coerce')
txn_nulls = df['transactiondate'].isna().sum()
txn_pct = (txn_nulls / total_rows) * 100
print(f"   - TransactionDate convertido. Nulos: {txn_nulls:,} ({txn_pct:.2f}%)")

# 4.3 Limpiar CustGender - Mantener nulos como están para no sesgar
gender_nulls = df['custgender'].isna().sum()
gender_pct = (gender_nulls / total_rows) * 100
print(f"   - Género con nulos: {gender_nulls:,} ({gender_pct:.2f}%)")

# 4.4 Asegurar tipos numéricos correctos
df['custaccountbalance'] = pd.to_numeric(df['custaccountbalance'], errors='coerce')
df['transactionamount'] = pd.to_numeric(df['transactionamount'], errors='coerce')

balance_nulls = df['custaccountbalance'].isna().sum()
balance_pct = (balance_nulls / total_rows) * 100
amount_nulls = df['transactionamount'].isna().sum()
amount_pct = (amount_nulls / total_rows) * 100

print(f"   - Balance nulos: {balance_nulls:,} ({balance_pct:.2f}%)")
print(f"   - Amount nulos: {amount_nulls:,} ({amount_pct:.2f}%)")

In [ ]:
# 4.5 Resumen de limpieza
print("\n📊 Resumen post-limpieza:")
print(df.info())
print(f"\n📈 Estadísticas descriptivas:")
df.describe()

## 5. Subir a BigQuery

In [ ]:
print("\n☁️ Subiendo a BigQuery...")

# Crear cliente de BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Configurar el esquema explícitamente para evitar problemas de tipos
schema = [
    bigquery.SchemaField("transactionid", "STRING"),
    bigquery.SchemaField("customerid", "STRING"),
    bigquery.SchemaField("customerdob", "DATE"),
    bigquery.SchemaField("custgender", "STRING"),
    bigquery.SchemaField("custlocation", "STRING"),
    bigquery.SchemaField("custaccountbalance", "FLOAT64"),
    bigquery.SchemaField("transactiondate", "DATE"),
    bigquery.SchemaField("transactiontime", "INT64"),
    bigquery.SchemaField("transactionamount", "FLOAT64"),
]

In [ ]:
# Configurar el job
table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Sobreescribir
)

# Subir DataFrame
job = client.load_table_from_dataframe(
    df, 
    table_ref, 
    job_config=job_config
)

# Esperar a que termine
job.result()

# Verificar
table = client.get_table(table_ref)
print(f"\n✅ Tabla creada exitosamente en BigQuery!")
print(f"   📍 Ubicación: {table_ref}")
print(f"   📊 Filas cargadas: {table.num_rows:,}")
print(f"   💾 Tamaño: {table.num_bytes / 1e6:.2f} MB")

## 6. Verificación Rápida

In [ ]:
query = f"""
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT customerid) as unique_customers,
    MIN(transactiondate) as first_date,
    MAX(transactiondate) as last_date,
    AVG(transactionamount) as avg_amount
FROM `{table_ref}`
"""

result = client.query(query).to_dataframe()
print("\n📋 Verificación de datos en BigQuery:")
result

In [ ]:
print("\n🎉 ¡Proceso completado exitosamente!")
print("\nPróximos pasos:")
print("1. Ejecuta el SQL '02_customer_features.sql' en BigQuery")
print("2. Continúa con el notebook de EDA")